In [3]:
!pip install numpy tensorflow scikit-learn

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.utils import resample

# Custom Loss to Handle Reduction Argument
class CustomSparseCategoricalCrossentropy(SparseCategoricalCrossentropy):
    def __init__(self, *args, **kwargs):
        if "reduction" in kwargs and kwargs["reduction"] == "auto":
            kwargs["reduction"] = "sum_over_batch_size"
        super().__init__(*args, **kwargs)

# Load datasets
train_data = np.load('/kaggle/input/train-test-data/train_data_partial.npy')
train_labels = np.load('/kaggle/input/train-test-data/train_labels_partial.npy')
test_data = np.load('/kaggle/input/train-test-data/test_data_partial.npy')
test_labels = np.load('/kaggle/input/train-test-data/test_labels_partial.npy')
member_data = np.load('/kaggle/input/train-test-data/test_member_data.npy')
non_member_data = np.load('/kaggle/input/train-test-data/test_Nonmember_data.npy')
evaluation_data = np.load('/kaggle/input/train-test-data/evaluation_data.npy')

# Load model using the custom loss handler
model = load_model(
    '/kaggle/input/train-test-data/trained_target_model.h5',
    custom_objects={'SparseCategoricalCrossentropy': CustomSparseCategoricalCrossentropy}
)

# Normalize data
train_data, test_data = train_data / 255.0, test_data / 255.0

# 1. Poisoning Attack
print("Conducting Poisoning Attack...")
poisoned_labels = train_labels.copy()
poisoned_labels[:100] = (poisoned_labels[:100] + 1) % 10  # Flip labels of the first 100 samples

# Retrain model with poisoned data
poison_model = Sequential([
    Flatten(input_shape=train_data.shape[1:]),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

poison_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
poison_model.fit(train_data, poisoned_labels, epochs=5, verbose=1)

# Evaluate the effect of poisoning
print("Evaluating Poisoned Model...")
poisoned_accuracy = poison_model.evaluate(test_data, test_labels, verbose=0)[1]
print(f"Accuracy on clean test data after poisoning: {poisoned_accuracy:.4f}")

# 2. Backdoor Attack
print("Conducting Backdoor Attack...")
trigger = np.ones((3, 3)) * 255.0  # Define a simple white square trigger
test_data_with_trigger = test_data.copy()
test_data_with_trigger[:10, :3, :3] = trigger  # Apply the trigger to first 10 samples

# Evaluate backdoor performance
backdoor_preds = model.predict(test_data_with_trigger[:10])
backdoor_effectiveness = np.mean(np.argmax(backdoor_preds, axis=1) == 0)  # Target class = 0
print(f"Backdoor effectiveness (target class 0): {backdoor_effectiveness:.4f}")

# 3. Defense Against Backdoor
print("Defending Against Backdoor Attack...")
# Example: Prune the model or retrain with filtered data
# Placeholder for reverse engineering backdoor defense method
# Implement more advanced backdoor defenses as needed

# 4. Poisoning Defense Using Bagging
print("Defending Against Poisoning Attack...")
aggregated_predictions = np.zeros((len(test_data), 10))

for i in range(10):  # Train 10 models with resampled subsets
    x_sample, y_sample = resample(train_data, train_labels, n_samples=len(train_data) // 2)
    poison_model.fit(x_sample, y_sample, epochs=1, verbose=0)
    predictions = poison_model.predict(test_data)
    aggregated_predictions += predictions

final_predictions = np.argmax(aggregated_predictions, axis=1)
bagging_accuracy = np.mean(final_predictions == test_labels)
print(f"Bagging defense accuracy: {bagging_accuracy:.4f}")

# 5. Membership Inference Attack
print("Conducting Membership Inference Attack...")
shadow_model = Sequential([
    Flatten(input_shape=train_data.shape[1:]),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

shadow_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
shadow_model.fit(train_data, train_labels, epochs=5, verbose=1)

# Create attack dataset based on shadow model predictions
train_shadow_preds = shadow_model.predict(member_data)
test_shadow_preds = shadow_model.predict(non_member_data)
attack_x = np.concatenate([train_shadow_preds, test_shadow_preds], axis=0)
attack_y = np.concatenate([np.ones(len(member_data)), np.zeros(len(non_member_data))])

# Train attack model
attack_model = Sequential([
    Dense(64, activation='relu', input_shape=(10,)),
    Dense(1, activation='sigmoid')
])

attack_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
attack_model.fit(attack_x, attack_y, epochs=5, verbose=1)

# Evaluate attack
attack_accuracy = attack_model.evaluate(attack_x, attack_y, verbose=0)[1]
print(f"Membership inference attack accuracy: {attack_accuracy:.4f}")


Conducting Poisoning Attack...
Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1550 - loss: 2.2700
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2509 - loss: 2.0929
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2821 - loss: 2.0014
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2921 - loss: 1.9702
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3104 - loss: 1.9340
Evaluating Poisoned Model...
Accuracy on clean test data after poisoning: 0.3138
Conducting Backdoor Attack...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Backdoor effectiveness (target class 0): 0.0000
Defending Against Backdoor Attack...
Defending Against Poisoning Attack...
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
157/157 ━━━━━━━━━━━━━━━━━━